In [1]:
import os
import os.path
import pickle
import shutil
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from os.path import join, exists
import numpy as np
from tqdm import tqdm
import random
%matplotlib inline

In [2]:
with open('results/ood_prob_dict.pickle', 'rb') as f:
    ood_prob_dict = pickle.load(f)

In [3]:
with open('results/HR_prob_dict.pickle', 'rb') as f:
    HR_prob_dict = pickle.load(f)

# Generate anchor image dict for deriving image pair (ref + target) dataset

A "anchor_images_dict" maps a target image in a sequence to all its reference images in this sequence. This is needed to be run before deploying the LR model.

In [7]:
dir_list = ['demo_sequences']

In [8]:
root_data_dir = 'data/sequences'

In [9]:
HR_threshold = 0.85

In [10]:
def anchor_model_1_get_all_images(prob_dict, fname_list, threshold=0.5):
    """
    prob_dict: {fname: probability}, e.g. {'2006_0.png': 0.001, '2007_1.png': 0.132, ...}
    fname_list: a candidate list of filenames
    """
    anchor_images = []
    for fname in fname_list:
        prob = prob_dict[fname]
        if prob >= threshold:
            anchor_images.append(fname)
    if len(anchor_images) == 0:
        anchor_images.append(fname_list[-1])
    return anchor_images

In [11]:
anchor_images_dict = dict()
for subdir in dir_list:
    data_dir = join(root_data_dir, subdir)
    for folder in os.listdir(data_dir):
        idx = folder.split('_')[0]
        folder_dir = join(data_dir, folder)
        fname_list = []
        for f in os.listdir(folder_dir):
            if f[-4:] == '.png':
                fname_list.append(f)
        HR_prob_dict_sub = HR_prob_dict[idx]
        anchor_images = anchor_model_1_get_all_images(HR_prob_dict_sub, fname_list, threshold=HR_threshold)
        anchor_images_dict[idx] = anchor_images

In [12]:
with open('results/anchor_images_dict.pickle', 'wb') as f:
    pickle.dump(anchor_images_dict, f)